In [15]:
# from backtesting.test import GOOG 

from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince

from Data_Binance import Crypto

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import talib

In [2]:
ticker = 'ETHUSDT'
start_date = '2024-09-20'
end_date = '2024-09-25'
interval = '1m' #1m -> minutes; h -> hours; d -> days; w -> weeks; M -> months
# infos = [ticker, start_date, end_date, interval]

crypto = Crypto(ticker, start_date, end_date, interval)


In [3]:
ETH = crypto.get_data()

In [4]:
ETH.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'dv', 'num_trades',
       'taker_buy_vol', 'taker_buy_base_vol'],
      dtype='object')

In [5]:
ETH.interpolate(inplace = True)

In [6]:
ETH_data = ETH[['Open', 'High', 'Low', 'Close', 'Volume']]

In [7]:
ETH_data

,Open,High,Low,Close,Volume
Close_time,,,,,
2024-09-20 00:01:00,2465.21,2466.67,2465.20,2466.67,131.4708
2024-09-20 00:02:00,2466.67,2467.16,2466.17,2466.66,116.9774
2024-09-20 00:03:00,2466.65,2467.17,2466.40,2467.16,70.5308
2024-09-20 00:04:00,2467.16,2467.16,2464.19,2464.24,230.9532
2024-09-20 00:05:00,2464.24,2466.19,2463.73,2466.19,268.8074
...,...,...,...,...,...
2024-09-24 23:57:00,2654.20,2654.70,2653.59,2654.19,73.3455
2024-09-24 23:58:00,2654.20,2655.07,2654.20,2654.65,62.7538
2024-09-24 23:59:00,2654.65,2654.65,2653.00,2653.13,61.7841


In [8]:
def optim_func(series) :
    if series["# Trades"] < 10:
        return -1
    return series[ "Equity Final [$]"] / series["Exposure Time [%]"]

In [9]:

class Rsioscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_window = 14
    def init(self):
        self.daily_rsi = self.I(talib.RSI, self.data.Close, self. rsi_window)
    def next(self) :
        price = self.data.Close[-1]

        if crossover (self.daily_rsi, self.upper_bound):
            self.position.close()
        elif self. lower_bound > self.daily_rsi[-1]:
            self.buy (size=1)


In [10]:
bt = Backtest(ETH_data,Rsioscillator, cash = 10_000)
stats = bt.run()

In [11]:
bt.plot()



/Users/theomontant/anaconda3/envs/tfenv/lib/python3.12/site-packages/backtesting/_plotting.py:460: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p1367', ...)

In [12]:
crypto.save_data(ETH)

In [17]:
df = pd.read_hdf('/Users/theomontant/Quant-project/Backtesting/Time_series/ETHUSDT_1m_2024-09-20_2024-09-25.h5', key='ohlcv')


In [18]:
df

,Open,High,Low,Close,Volume,dv,num_trades,taker_buy_vol,taker_buy_base_vol
2024-09-20 00:01:00,2465.21,2466.67,2465.20,2466.67,131.4708,324217.484686,1770.0,73.7812,181949.329439
2024-09-20 00:02:00,2466.67,2467.16,2466.17,2466.66,116.9774,288546.274681,2079.0,61.3501,151321.641780
2024-09-20 00:03:00,2466.65,2467.17,2466.40,2467.16,70.5308,173997.823044,1657.0,33.4646,82555.032926
2024-09-20 00:04:00,2467.16,2467.16,2464.19,2464.24,230.9532,569302.984217,3305.0,154.8455,381651.866285
2024-09-20 00:05:00,2464.24,2466.19,2463.73,2466.19,268.8074,662617.759862,2147.0,249.5442,615132.765101
...,...,...,...,...,...,...,...,...,...
2024-09-24 23:57:00,2654.20,2654.70,2653.59,2654.19,73.3455,194663.011606,1319.0,52.2566,138690.709003
2024-09-24 23:58:00,2654.20,2655.07,2654.20,2654.65,62.7538,166599.507935,1048.0,42.3049,112310.157332
2024-09-24 23:59:00,2654.65,2654.65,2653.00,2653.13,61.7841,163970.746385,605.0,3.3464,8882.998616
2024-09-25 00:00:00,2653.13,2653.20,2652.13,2653.20,152.7205,405096.344149,1123.0,40.8457,108336.346960
